In [2]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
%matplotlib inline
import pandas_datareader.data as web


In [3]:
from dotenv import load_dotenv
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2') 

In [4]:
assets = api.list_assets()
tradeable = [asset for asset in assets if asset.tradable]


In [5]:
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets])
asset_info_df.head()

,symbol
0,BKAG
1,BKEM
2,BKHY
3,BKIE
4,BKSB


In [7]:
main_df = pd.DataFrame()
screened_stocks_df = pd.read_csv('../Data/screened_stocks.csv')
screened_stocks_tickers = screened_stocks_df['Ticker']
tickers= screened_stocks_tickers.tolist()
#tickers = ['TSLA','SPY']
timeframe = '1D'
end_date = datetime.now()
start_date = end_date + timedelta(-30)

for count, ticker in enumerate(tickers):
    df = api.get_barset(
        ticker,
        timeframe,
        limit=None,
        start=start_date,
        end=end_date,
        after=None,
        until=None,
    ).df
    df.index = df.index.date
    series = df[ticker]['close']
    series.rename(ticker, inplace =True)
    if main_df.empty:
        main_df = series
    else:
        main_df = pd.concat([main_df,series], axis = 'columns',join='inner')
main_df.head()

sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.marke

,AAWW,AAXN,ABCB,ABTX,ACBI,ACGL,ACHC,ACIA,ACLS,ACMR,...,YJ,YNDX,YORW,YY,ZIXI,ZNGA,ZS,ZUMZ,ZVO,ZYXI
2020-04-02,23.48,62.51,22.92,23.04,10.81,26.30,17.77,66.840,17.24,27.26,...,3.46,32.61,41.00,54.63,3.61,6.690,61.12,14.51,1.50,10.57
2020-04-06,21.95,67.71,22.32,23.32,11.44,27.03,17.68,66.660,19.01,32.51,...,3.25,33.87,43.71,59.87,4.00,6.810,63.28,17.05,1.48,11.13
2020-04-07,23.11,69.01,22.67,22.96,10.98,27.89,18.43,67.215,18.81,33.24,...,3.21,34.46,40.18,59.96,4.37,6.565,62.36,19.27,1.45,11.87
2020-04-09,25.27,74.70,26.57,27.13,12.42,30.99,22.97,67.200,19.57,32.98,...,3.25,36.13,43.05,57.05,5.06,6.915,62.59,21.88,1.55,13.00
2020-04-13,24.15,73.50,24.96,25.28,11.54,29.23,21.85,66.760,19.64,33.27,...,3.23,36.70,42.65,56.58,5.39,7.050,62.17,21.25,1.54,12.87


,AAWW,AAXN,ABCB,ABTX,ACBI,ACGL,ACHC,ACIA,ACLS,ACMR,...,YJ,YNDX,YORW,YY,ZIXI,ZNGA,ZS,ZUMZ,ZVO,ZYXI
2020-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-06,-0.065162,0.083187,-0.026178,0.012153,0.058279,0.027757,-0.005065,-0.002693,0.102668,0.192590,...,-0.060694,0.038638,0.066098,0.095918,0.108033,0.017937,0.035340,0.175052,-0.013333,0.052980
2020-04-07,0.052847,0.019200,0.015681,-0.015437,-0.040210,0.031817,0.042421,0.008326,-0.010521,0.022455,...,-0.012308,0.017420,-0.080760,0.001503,0.092500,-0.035977,-0.014539,0.130205,-0.020270,0.066487
2020-04-09,0.093466,0.082452,0.172034,0.181620,0.131148,0.111151,0.246337,-0.000223,0.040404,-0.007822,...,0.012461,0.048462,0.071429,-0.048532,0.157895,0.053313,0.003688,0.135444,0.068966,0.095198
2020-04-13,-0.044321,-0.016064,-0.060595,-0.068190,-0.070853,-0.056793,-0.048759,-0.006548,0.003577,0.008793,...,-0.006154,0.015776,-0.009292,-0.008238,0.065217,0.019523,-0.006710,-0.028793,-0.006452,-0.010000
2020-04-14,0.059627,0.038503,-0.034856,-0.023734,-0.000867,0.001026,0.075973,0.014829,0.111507,0.035768,...,-0.018576,0.004360,0.011958,0.024214,0.105751,0.021631,0.069165,0.011765,-0.012987,0.034965
2020-04-16,-0.063306,-0.048081,-0.161478,-0.126418,-0.105811,-0.095010,-0.044236,0.003100,-0.041228,0.035984,...,-0.025237,-0.051004,-0.024560,0.059534,0.041946,0.077057,0.060328,-0.053023,0.125000,-0.070571
2020-04-23,0.161869,-0.027388,0.075248,0.040816,0.076625,-0.083837,-0.028482,0.007431,0.026278,0.003922,...,0.113269,0.007433,-0.010214,-0.021173,-0.152979,-0.038350,-0.060159,-0.088409,-0.099415,0.126817


In [9]:
#THIS is the only thing that worked for me to get closing stock information
sp500 = web.DataReader(['sp500'], 'fred', start_date, end_date)
sp500.head()

,sp500
DATE,
2019-04-30,2945.83
2019-05-01,2923.73
2019-05-02,2917.52
2019-05-03,2945.64
2019-05-06,2932.47


In [12]:
stocks = ['ABTX', 'QQQ', 'IWM', 'AAPL', 'FB', 'GDX']
read_stocks = web.DataReader(stocks, 'yahoo',start_date,end_date)
close_stocks = read_stocks['Close']
close_stocks.head()

Symbols,ABTX,QQQ,IWM,AAPL,FB,GDX
Date,,,,,,
2019-04-30,34.570000,189.539993,158.289993,200.669998,193.399994,20.889999
2019-05-01,34.480000,188.929993,156.789993,210.520004,193.029999,20.520000
2019-05-02,34.900002,188.110001,157.520004,209.149994,192.529999,20.170000
2019-05-03,35.820000,191.110001,160.529999,211.750000,195.470001,20.290001
2019-05-06,35.959999,189.940002,160.710007,208.479996,193.880005,20.299999


In [11]:
sp500_returns = sp500.pct_change()
sp500_returns.dropna(inplace = True)
sp500_returns.head()

,sp500
DATE,
2019-05-01,-0.007502
2019-05-02,-0.002124
2019-05-03,0.009638
2019-05-06,-0.004471
2019-05-07,-0.016512


In [13]:
stocks_returns = close_stocks.pct_change()
stocks_returns.dropna(inplace=True)
stocks_returns.head()

Symbols,ABTX,QQQ,IWM,AAPL,FB,GDX
Date,,,,,,
2019-05-01,-0.002603,-0.003218,-0.009476,0.049086,-0.001913,-0.017712
2019-05-02,0.012181,-0.004340,0.004656,-0.006508,-0.002590,-0.017057
2019-05-03,0.026361,0.015948,0.019109,0.012431,0.015270,0.005949
2019-05-06,0.003908,-0.006122,0.001121,-0.015443,-0.008134,0.000493
2019-05-07,-0.005562,-0.019480,-0.019787,-0.026957,-0.021199,0.020197


In [14]:
#getting covariance for the stocks with sp500
def get_covariance():
    covariance_dict = {}
    for symbol in stocks_returns:
        covariance = stocks_returns[symbol].cov(sp500_returns['sp500'])
        covariance_dict.update({symbol: covariance})
    return covariance_dict
get_covariance()

{'ABTX': 0.00048221483992885153,
 'QQQ': 0.00040274501767225963,
 'IWM': 0.000423688887824185,
 'AAPL': 0.0004666859723339267,
 'FB': 0.00040242117319952013,
 'GDX': 8.182855353608699e-05}

In [17]:
#getting variance 
def get_variance():
    variance_dict = {}
    for symbol in stocks_returns:
        variance = stocks_returns[symbol].var()
        variance_dict.update({symbol: variance})
    return variance_dict
get_variance()


{'SPY': 0.00039149070130194266,
 'QQQ': 0.0004222483922603792,
 'IWM': 0.000496277754655345,
 'AAPL': 0.0006646420682138242,
 'FB': 0.000594485107578183,
 'GDX': 0.001218102328831449}

In [18]:
#calculating beta
def get_beta():
    beta_dict = {}
    for symbol in stocks_returns:
        covariance = stocks_returns[symbol].cov(sp500_returns['sp500'])
        variance = stocks_returns[symbol].var()
        beta = covariance/variance
        beta_dict.update({symbol: beta})
    return beta_dict

get_beta()

{'SPY': 1.025215898301114,
 'QQQ': 0.9538106599205407,
 'IWM': 0.8537333858907872,
 'AAPL': 0.7021613506772303,
 'FB': 0.676923892742925,
 'GDX': 0.06717707667022263}

-0.03146226605982977